In [1]:
import os
from monai.data import load_decathlon_datalist, Dataset

os.listdir('/data/')

['LUNA16_datasplit', 'LUNA16_Images_resample']

In [2]:
base_dir="/data/LUNA16_Images_resample/"
downloaded_datasplit_dir='/data/LUNA16_datasplit'
data_list_file_path = os.path.join(downloaded_datasplit_dir, "mhd_original/dataset_fold0.json")

In [3]:
inference_data = load_decathlon_datalist(
    data_list_file_path,
    is_segmentation=True,
    data_list_key="validation",
    base_dir=base_dir
)
inference_ds = Dataset(
    data=inference_data,
    transform=inference_transforms
)


In [4]:
inference_data

[{'box': [[110.1355596,
    -32.0169515,
    -241.1046505,
    6.005399015,
    6.005399015,
    6.005399015]],
  'image': '/data/LUNA16_Images_resample/1.3.6.1.4.1.14519.5.2.1.6279.6001.281489753704424911132261151767.mhd',
  'label': [0]},
 {'box': [[68.05574564,
    43.7031034,
    -266.0215516,
    12.68580657,
    12.68580657,
    12.68580657]],
  'image': '/data/LUNA16_Images_resample/1.3.6.1.4.1.14519.5.2.1.6279.6001.313334055029671473836954456733.mhd',
  'label': [0]},
 {'box': [[-36.55628288,
    94.95448429,
    -231.0861394,
    8.208342207,
    8.208342207,
    8.208342207]],
  'image': '/data/LUNA16_Images_resample/1.3.6.1.4.1.14519.5.2.1.6279.6001.333145094436144085379032922488.mhd',
  'label': [0]},
 {'box': [[-112.7948634,
    -227.3130252,
    -199.2932673,
    6.21739005,
    6.21739005,
    6.21739005]],
  'image': '/data/LUNA16_Images_resample/1.3.6.1.4.1.14519.5.2.1.6279.6001.640729228179368154416184318668.mhd',
  'label': [0]},
 {'box': [[-104.7330571,
    -65.2803